In [1]:
import sklearn
import pandas as pd
import pymorphy2

In [2]:
p_data = pd.read_csv('data/positive.csv', sep=';', header=None)
n_data = pd.read_csv('data/negative.csv', sep=';', header=None)

dataset = pd.concat([p_data, n_data])
dataset = dataset[[3, 4]]

dataset.columns = ['text', 'label']

morph = pymorphy2.MorphAnalyzer()

In [4]:
def text_cleaner(text):
    # к нижнему регистру
    text = text.lower()
    
    # оставляем в предложении только русские буквы (таким образом
    # удалим и ссылки, и имена пользователей, и пунктуацию и т.д.)
    alph = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    
    cleaned_text = ''
    for char in text:
        if (char.isalpha() and char[0] in alph) or (char == ' '):
            cleaned_text += char
        
    result = []
    for word in cleaned_text.split():
        # лемматизируем
        result.append(morph.parse(word)[0].normal_form)
                              
    return ' '.join(result)

In [6]:
stop_symbols_count = []
for text in dataset['text']:
    stop_symbols_count.append(count_of_stop_symbols(text, stop_symbols))

stop_symbols_count = pd.DataFrame(stop_symbols_count, columns=stop_symbols)

dataset = pd.concat([dataset, stop_symbols_count],
                    axis=1, join='inner')

In [7]:
dataset.head()

,text,label,repeat,favourite,:),),:D,:(,(,:D,!
0,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,1,1,0,0,1,0
1,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,0,1,0,0,1,0
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1,0,0,0,0,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,0,1,0,0,1,0
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,0,1,0,0,1,0


In [ ]:
dataset['text'] = dataset['text'].apply(text_cleaner)

dataset.to_csv('data/cleaned_data.csv')